## 14.1 ロジスティック回帰
```statsmodels```と```sklearn```を用いたロジスティック回帰のスニペット。


### (共通)データの読み込み
```seaborn```の```tips```データセットを用いる。

In [ ]:
import polars as pl
import seaborn as sns

# データの読み込み
titanic = pl.DataFrame(sns.load_dataset("titanic"))

# 解析に使用するデータを抽出する
titanic_sub = (
    titanic[["survived", "sex", "age", "embarked"]].drop_nulls()
)
# ダミー変数作成時にpandasでは自動的にsortが行われるがpolarsでは行われないので、順番を入れかえておく。
titanic_sub = titanic_sub.sort(by = ["sex", "age", "embarked"])
display(titanic_sub.head())

# 生存者数
display(titanic_sub["survived"].value_counts())
# 乗船場所
display(titanic_sub["embarked"].value_counts())

### 14.1.1 statsmodels

In [ ]:
import statsmodels.formula.api as smf

# モデルを学習する
form = "survived ~ sex + age + embarked"
model = smf.logit(formula = form, data = titanic_sub).fit()

# 学習結果を表示する
display(model.summary())

import pandas as pd
import numpy as np
res_sm = pd.DataFrame(model.params, columns = ["coefs_sm"])
res_sm["odds_sm"] = np.exp(res_sm["coefs_sm"])
display(res_sm.round(3))

### 14.1.2 sklearn

In [ ]:
from sklearn import linear_model

# ダミー変数を作成する
import polars.selectors as cs
# 数値型は外しておく
df_numeric = titanic_sub.select(cs.numeric()).drop("survived")
# カテゴリ変数のみダミー変数を作成する
df_categorical = titanic_sub.select(cs.string())
df_dummy = df_categorical.to_dummies(drop_first = True)
# 学習用のDataFrameを作成する
df_study = pl.concat(items = [df_numeric, df_dummy], how = "horizontal")


# モデルを学習する
lr = linear_model.LogisticRegression(penalty = None)
model = lr.fit(X = df_study, y = titanic_sub["survived"])


# 学習結果を表示する
import numpy as np
# 係数の名称
labels = ["Intercept"]
for col in df_study.columns:
    labels.append(col)
# 係数
coefficients = np.append(
    model.intercept_, model.coef_
)
# 学習結果
result = pl.DataFrame({
    "Label": labels,
    "Coefficient": coefficients
})
# オッズ
result = result.with_columns(
    result["Coefficient"].exp().round(decimals = 3).alias("Odds")
)
display(result)